# **Demand Forecasting for Ride-Sharing Trips using Time Series Analysis**

## Golas:
1. Prediksi pola demand trip per ride-sharing platform\
tujuannya yaitu melakukan forcasting permintaan trip/jumlah trip per jam dan per hari untuk bisa menyeimbangkan kebutuhan penumpang dengan jumlah driver

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
pd.options.display.max_columns = None

In [2]:
PATH_DATA_RAW= '../Dataset/Raw/'
PATH_DATA_CLEAN = '../Dataset/Clean/'

# Data Definition

In [3]:
calendar = pd.read_csv(PATH_DATA_RAW+'Calendar.csv')
city = pd.read_csv(PATH_DATA_RAW+'city.csv')
passenger = pd.read_csv(PATH_DATA_RAW+'passenger.csv')
trip = pd.read_csv(PATH_DATA_RAW+'trip.csv')

### **Calendar Table**

In [4]:
print(calendar['id'].nunique(), calendar.shape, calendar['calendar_date'].nunique())
print(calendar['holiday'].value_counts())
calendar.tail()

1096 (1096, 4) 1096
0    1061
1      35
Name: holiday, dtype: int64


,id,calendar_date,week_day,holiday
1091,47815,12/27/2020,Sunday,0
1092,47816,12/28/2020,Monday,0
1093,47817,12/29/2020,Tuesday,0
1094,47818,12/30/2020,Wednesday,0
1095,47819,12/31/2020,Thursday,0


Deskripsi:
* id: id unik untuk setiap tanggal
* calendar_date: tanggal dalam format MM/DD/YYYY
* week_day: nama-nama hari dalam seminggu, senin-minggu
* holiday: apakah tanggal tersebut adalah libur (1) atau tidak (0), total ada 35 hari libur

In [5]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1096 non-null   int64 
 1   calendar_date  1096 non-null   object
 2   week_day       1096 non-null   object
 3   holiday        1096 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 34.4+ KB


In [6]:
# Menambahkan kolom is_weekend untuk analisis tambahan
calendar['is_weekend'] = calendar['week_day'].apply(lambda day: 1 if day.lower() in ['saturday', 'sunday'] else 0)

In [13]:
calendar.calendar_date = pd.to_datetime(calendar.calendar_date)
calendar.rename(columns={'id': 'date_id'}, inplace=True)
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date_id        1096 non-null   int64         
 1   calendar_date  1096 non-null   datetime64[ns]
 2   week_day       1096 non-null   object        
 3   holiday        1096 non-null   int64         
 4   is_weekend     1096 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 42.9+ KB


In [31]:
joblib.dump(calendar, PATH_DATA_CLEAN+'calendar.pkl')

['../Dataset/Clean/calendar.pkl']

### **City Table**

In [7]:
city

,id,name
0,42868,Mordor
1,42948,Erebor
2,43006,Minas Tirith
3,43020,Isengard
4,43050,Gondor
5,43094,Moria
6,43116,Rohan
7,43179,Rivendell
8,43244,Minas Ithis
9,43278,The Shire


Deskripsi:
* id: id unik untuk setiap kota
* name: nama kota. Total ada 10 kota pada dataset ini

In [8]:
city.rename(columns={'id': 'city_id'}, inplace=True)
city.rename(columns={'name': 'city'}, inplace=True)
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city_id  10 non-null     int64 
 1   city     10 non-null     object
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


In [9]:
joblib.dump(city, PATH_DATA_CLEAN+'city.pkl')

['../Dataset/Clean/city.pkl']

### **Passenger Table**

In [10]:
print(passenger.shape)
passenger.head(10)

(1235782, 2)


,id,first_call_time
0,00000fab,5/11/2019 8:23:54 PM
1,0000149d,12/24/2018 9:21:33 PM
2,00002b66,9/20/2018 9:08:24 PM
3,00002bc7,7/28/2019 11:27:56 AM
4,0000332e,2/21/2019 11:34:24 PM
5,00003af1,11/3/2017 4:44:59 AM
6,000048aa,8/2/2019 8:01:07 AM
7,0000497f,7/11/2019 3:39:47 AM
8,00004a41,3/9/2018 9:10:11 AM
9,000070ff,5/4/2019 5:29:53 PM


In [11]:
passenger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235782 entries, 0 to 1235781
Data columns (total 2 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   id               1235782 non-null  object
 1   first_call_time  1222460 non-null  object
dtypes: object(2)
memory usage: 18.9+ MB


Deskripsi:
* id : id unik untuk setiap penumpang
* first_call_time : waktu/tanggal pemesanan perjalanan pertama

Total ada 1.235.782 penumpang

In [12]:
passenger.first_call_time = pd.to_datetime(passenger.first_call_time, format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
passenger.rename(columns={'id': 'passenger_id'}, inplace=True)
passenger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235782 entries, 0 to 1235781
Data columns (total 2 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   passenger_id     1235782 non-null  object        
 1   first_call_time  1209116 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 18.9+ MB


### **Trip Table**

In [13]:
print(trip['id'].nunique(),trip['passenger_id'].nunique(),trip['driver_id'].nunique(), trip.shape)
trip.sample(5)

2318357 1195765 145611 (2318357, 9)


,id,driver_id,passenger_id,city_id,call_time,finish_time,surge_rate,trip_distance,trip_fare
1353522,a2dbb5a0,13ce48b5,afb8b053,43006,6/3/2019 6:24:21 PM,6/3/2019 6:36:47 PM,0.0,2.39,2.51
608755,aa92655b,c195ef43,750a14f9,43020,6/29/2019 3:22:16 AM,6/29/2019 3:33:16 AM,0.0,3.26,1.44
2092017,6b27afdf,b2a27a37,6955891c,43116,6/6/2019 7:28:41 PM,6/6/2019 7:42:19 PM,0.0,3.37,3.10
1796423,1b1b8349,1bd571c7,d2587732,43179,8/1/2019 7:34:30 AM,8/1/2019 7:55:39 AM,0.0,6.97,3.06
1596405,e4e25981,a1244cf9,10bdb794,43006,7/26/2019 1:08:26 AM,7/26/2019 1:20:47 AM,0.0,4.67,2.11


Deskripsi:
* id: id unik untuk setiap perjalanan
* driver_id: id pengemudi
* passenger_id: id penumpang
* city_id: asal kota perjalanan
* call_time: waktu pemesanan perjalanan
* finish_time: waktu perjalanan selesai
* surge_rate: nilai kenaikan tarif dinamis pada saat peak hours
* trip_distance: jarak tempuh dalam km
* trip_fare: biaya/tarif perjalanan

In [14]:
trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318357 entries, 0 to 2318356
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             object 
 1   driver_id      object 
 2   passenger_id   object 
 3   city_id        int64  
 4   call_time      object 
 5   finish_time    object 
 6   surge_rate     float64
 7   trip_distance  float64
 8   trip_fare      float64
dtypes: float64(3), int64(1), object(5)
memory usage: 159.2+ MB


In [15]:
trip.call_time = pd.to_datetime(trip.call_time, format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
trip.finish_time = pd.to_datetime(trip.finish_time, format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318357 entries, 0 to 2318356
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   id             object        
 1   driver_id      object        
 2   passenger_id   object        
 3   city_id        int64         
 4   call_time      datetime64[ns]
 5   finish_time    datetime64[ns]
 6   surge_rate     float64       
 7   trip_distance  float64       
 8   trip_fare      float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(3)
memory usage: 159.2+ MB


In [16]:
trip['call_time_date_only'] = trip['call_time'].dt.date
trip['call_time_date_only'] = pd.to_datetime(trip.call_time_date_only, format='%Y/%m/%d')
trip.rename(columns={'id': 'trip_id'}, inplace=True)
trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318357 entries, 0 to 2318356
Data columns (total 10 columns):
 #   Column               Dtype         
---  ------               -----         
 0   trip_id              object        
 1   driver_id            object        
 2   passenger_id         object        
 3   city_id              int64         
 4   call_time            datetime64[ns]
 5   finish_time          datetime64[ns]
 6   surge_rate           float64       
 7   trip_distance        float64       
 8   trip_fare            float64       
 9   call_time_date_only  datetime64[ns]
dtypes: datetime64[ns](3), float64(3), int64(1), object(3)
memory usage: 176.9+ MB


In [17]:
trip.head()

,trip_id,driver_id,passenger_id,city_id,call_time,finish_time,surge_rate,trip_distance,trip_fare,call_time_date_only
0,a84c0db2,cb64172e,2ea4359d,43006,2019-09-08 22:01:28,2019-09-08 22:15:54,0.3,1.92,4.36,2019-09-08
1,a84c2d10,3034e6fe,e1bd0c8d,43006,2019-07-01 20:56:28,2019-07-01 21:22:09,0.0,9.36,6.91,2019-07-01
2,a84c3e75,93fb6802,75e6d450,43050,2019-08-20 21:30:46,2019-08-20 21:46:48,0.0,4.39,3.48,2019-08-20
3,a84c3f6f,25fc9273,5ba48e89,43020,2019-09-08 02:42:41,2019-09-08 02:59:56,0.0,2.04,1.54,2019-09-08
4,a84c5ce2,484c4d0c,fdb2ef59,43006,2019-06-10 09:29:18,2019-06-10 09:55:36,0.0,7.37,5.83,2019-06-10


## Join All Table

In [18]:
df = pd.merge(trip, calendar, left_on='call_time_date_only', right_on='calendar_date', how='left')
df = pd.merge(df, passenger, left_on='passenger_id', right_on='passenger_id', how='left')
df = pd.merge(df, city, left_on='city_id', right_on='city_id', how='left')
df

,trip_id,driver_id,passenger_id,city_id,call_time,finish_time,surge_rate,trip_distance,trip_fare,call_time_date_only,date_id,calendar_date,week_day,holiday,is_weekend,first_call_time,city
0,a84c0db2,cb64172e,2ea4359d,43006,2019-09-08 22:01:28,2019-09-08 22:15:54,0.3,1.92,4.36,2019-09-08,47339,2019-09-08,Sunday,0,1,2018-08-18 10:56:35,Minas Tirith
1,a84c2d10,3034e6fe,e1bd0c8d,43006,2019-07-01 20:56:28,2019-07-01 21:22:09,0.0,9.36,6.91,2019-07-01,47270,2019-07-01,Monday,0,0,2019-04-16 23:50:42,Minas Tirith
2,a84c3e75,93fb6802,75e6d450,43050,2019-08-20 21:30:46,2019-08-20 21:46:48,0.0,4.39,3.48,2019-08-20,47320,2019-08-20,Tuesday,0,0,2018-08-17 01:51:01,Gondor
3,a84c3f6f,25fc9273,5ba48e89,43020,2019-09-08 02:42:41,2019-09-08 02:59:56,0.0,2.04,1.54,2019-09-08,47339,2019-09-08,Sunday,0,1,2019-05-10 05:35:30,Isengard
4,a84c5ce2,484c4d0c,fdb2ef59,43006,2019-06-10 09:29:18,2019-06-10 09:55:36,0.0,7.37,5.83,2019-06-10,47249,2019-06-10,Monday,0,0,2018-10-06 20:35:16,Minas Tirith
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318352,a84b949e,a454c95d,906eee37,43006,2019-07-06 06:32:54,2019-07-06 06:53:35,0.0,4.72,3.18,2019-07-06,47275,2019-07-06,Saturday,0,1,2018-09-16 05:31:02,Minas Tirith
2318353,a84bc1e0,1a154fe4,313bd7fe,43050,2019-07-22 17:58:25,2019-07-22 18:54:49,0.0,20.20,10.69,2019-07-22,47291,2019-07-22,Monday,0,0,2018-05-13 23:53:07,Gondor
2318354,a84bcd75,9e8e5eb2,47253b43,43050,2019-09-07 18:24:18,2019-09-07 18:47:46,0.0,8.78,7.41,2019-09-07,47338,2019-09-07,Saturday,1,1,2019-04-25 19:08:45,Gondor
2318355,a84bde9a,e609e0a3,b731ab38,43006,2019-07-02 18:24:59,2019-07-02 18:38:58,0.0,3.40,1.84,2019-07-02,47271,2019-07-02,Tuesday,0,0,2019-06-16 14:08:58,Minas Tirith


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2318357 entries, 0 to 2318356
Data columns (total 17 columns):
 #   Column               Dtype         
---  ------               -----         
 0   trip_id              object        
 1   driver_id            object        
 2   passenger_id         object        
 3   city_id              int64         
 4   call_time            datetime64[ns]
 5   finish_time          datetime64[ns]
 6   surge_rate           float64       
 7   trip_distance        float64       
 8   trip_fare            float64       
 9   call_time_date_only  datetime64[ns]
 10  date_id              int64         
 11  calendar_date        datetime64[ns]
 12  week_day             object        
 13  holiday              int64         
 14  is_weekend           int64         
 15  first_call_time      datetime64[ns]
 16  city                 object        
dtypes: datetime64[ns](5), float64(3), int64(4), object(5)
memory usage: 318.4+ MB


In [20]:
df.drop(columns='call_time_date_only',axis=1 ,inplace=True)

## Checking missing and duplicate value

In [21]:
# Function check total missing value from all row of table
def missing_value(table):
    total = 0
    for i in table.isna().sum():
        total = total+i
    return total

In [22]:
print("Total missing value:",missing_value(df))
print("Percentage missing value:",missing_value(df)/len(df)*100)
df.isna().sum()

Total missing value: 23245
Percentage missing value: 1.0026497213328232


trip_id                0
driver_id              0
passenger_id           0
city_id                0
call_time              0
finish_time            0
surge_rate             6
trip_distance      23239
trip_fare              0
date_id                0
calendar_date          0
week_day               0
holiday                0
is_weekend             0
first_call_time        0
city                   0
dtype: int64

Missing value hanya 1% dari total data keseluruhan, jadi missing value ini akan dihapus

In [23]:
df.dropna(inplace=True)

In [24]:
df.isna().sum()

trip_id            0
driver_id          0
passenger_id       0
city_id            0
call_time          0
finish_time        0
surge_rate         0
trip_distance      0
trip_fare          0
date_id            0
calendar_date      0
week_day           0
holiday            0
is_weekend         0
first_call_time    0
city               0
dtype: int64

In [25]:
print("Total duplicate value df:",df.duplicated().sum())

Total duplicate value df: 0


# Change Value data

In [26]:
# Mengubah tipe data object ke datetime
# Kolom yang memiliki bagian waktu (JJ:MM:DD) akan dianalisis untuk jamnya saja
df.call_time = pd.to_datetime(df.call_time).dt.floor('h')
df.finish_time = pd.to_datetime(df.finish_time).dt.floor('h')
df.calendar_date = pd.to_datetime(df.calendar_date)
df.first_call_time = pd.to_datetime(df.first_call_time).dt.floor('h')

In [27]:
# Kolom yang memiliki bagian waktu (JJ:MM:DD) akan dianalisis juga tanpa bagian waktunya
# Analisisnya akan menjadi harian
df['finish_time_date'] = pd.to_datetime(df.finish_time).dt.floor('d')
df['first_call_time_date'] = pd.to_datetime(df.first_call_time).dt.floor('d')

In [28]:
df

,trip_id,driver_id,passenger_id,city_id,call_time,finish_time,surge_rate,trip_distance,trip_fare,date_id,calendar_date,week_day,holiday,is_weekend,first_call_time,city,finish_time_date,first_call_time_date
0,a84c0db2,cb64172e,2ea4359d,43006,2019-09-08 22:00:00,2019-09-08 22:00:00,0.3,1.92,4.36,47339,2019-09-08,Sunday,0,1,2018-08-18 10:00:00,Minas Tirith,2019-09-08,2018-08-18
1,a84c2d10,3034e6fe,e1bd0c8d,43006,2019-07-01 20:00:00,2019-07-01 21:00:00,0.0,9.36,6.91,47270,2019-07-01,Monday,0,0,2019-04-16 23:00:00,Minas Tirith,2019-07-01,2019-04-16
2,a84c3e75,93fb6802,75e6d450,43050,2019-08-20 21:00:00,2019-08-20 21:00:00,0.0,4.39,3.48,47320,2019-08-20,Tuesday,0,0,2018-08-17 01:00:00,Gondor,2019-08-20,2018-08-17
3,a84c3f6f,25fc9273,5ba48e89,43020,2019-09-08 02:00:00,2019-09-08 02:00:00,0.0,2.04,1.54,47339,2019-09-08,Sunday,0,1,2019-05-10 05:00:00,Isengard,2019-09-08,2019-05-10
4,a84c5ce2,484c4d0c,fdb2ef59,43006,2019-06-10 09:00:00,2019-06-10 09:00:00,0.0,7.37,5.83,47249,2019-06-10,Monday,0,0,2018-10-06 20:00:00,Minas Tirith,2019-06-10,2018-10-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318352,a84b949e,a454c95d,906eee37,43006,2019-07-06 06:00:00,2019-07-06 06:00:00,0.0,4.72,3.18,47275,2019-07-06,Saturday,0,1,2018-09-16 05:00:00,Minas Tirith,2019-07-06,2018-09-16
2318353,a84bc1e0,1a154fe4,313bd7fe,43050,2019-07-22 17:00:00,2019-07-22 18:00:00,0.0,20.20,10.69,47291,2019-07-22,Monday,0,0,2018-05-13 23:00:00,Gondor,2019-07-22,2018-05-13
2318354,a84bcd75,9e8e5eb2,47253b43,43050,2019-09-07 18:00:00,2019-09-07 18:00:00,0.0,8.78,7.41,47338,2019-09-07,Saturday,1,1,2019-04-25 19:00:00,Gondor,2019-09-07,2019-04-25
2318355,a84bde9a,e609e0a3,b731ab38,43006,2019-07-02 18:00:00,2019-07-02 18:00:00,0.0,3.40,1.84,47271,2019-07-02,Tuesday,0,0,2019-06-16 14:00:00,Minas Tirith,2019-07-02,2019-06-16


# Export Data

In [30]:
joblib.dump(df, PATH_DATA_CLEAN+'clean_data.pkl')

['../Dataset/Clean/clean_data.pkl']